# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [374]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [375]:
# TODO: Import the necessary libs
# For example: 
import os
from dotenv import load_dotenv
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from pydantic import BaseModel
from typing import List, Optional
import chromadb
from datetime import datetime
from tavily import TavilyClient
from openai import OpenAI

In [376]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [377]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

class Game(BaseModel):
    Platform: str
    Name: str
    YearOfRelease: int
    Description: str
    SimilarityScore: float


@tool
def retrieve_game_tool(query: str) -> List[str]:
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains a dict containing:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    - SimilarityScore: How similar the results are to the query
    
    Returns: retrieved_docs"""
    
    print('Using Tool: **retrieve_game_tool**')
    print(f'Looking for documents to address query: {query}')
    result = collection.query(
        query_texts=[query],
        n_results=3,
        include=['documents','distances','metadatas'])
    print(f"{len(result['documents'][0])} documents found.")

    retrieved_docs =[]
    try:
        print(f"______________ Documents Related to: {query} _______________")
        
        for i, (d, dist, meta) in enumerate(zip(result['documents'][0],result['distances'][0],result['metadatas'][0])):           
            retrieved_docs.append(f"Platform: {meta['Platform']}\n, Name: {meta['Name']}\n, YearOfRelease: {meta['YearOfRelease']}\n, Description: {meta['Description']}\n, SimilarityScore: {round(1 - dist,2)}\n")
   
    
    except:
        None
    print(f'{retrieved_docs}')
    return {'answer':retrieved_docs}


#### Evaluate Retrieval Tool

In [378]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result

   
@tool
def evaluate_retrieval_tool(query: str, retrieved_docs: list[str]) -> dict:
    """Based on the user's query and on the list of retrieved documents, 
   it will analyze the usability of the documents to respond to that query. 
   args: 
   - query: original query from user
   - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
   The result includes:
   - useful: whether the documents are useful to answer the query
   - description: description about the evaluation result"""
   
    print('Using Tool **evaluate_retrieval_tool**')
    client = OpenAI(api_key=OPENAI_API_KEY, base_url="https://openai.vocareum.com/v1")
    

    eval_query = ("Your task is to evaluate if the documents are enough to respond the query. "
     "Give a detailed explanation, so it's possible to take an action to accept it or not."
     """Return Results in JSON Format, like so:
     {
                 "useful": true or false,
                 "description": "detailed explanation to help understand why it was or was not useful",
                 "relevant_info": "If useful = true then return the answer to the query here"

    }""")

    print(f"Evaluating {len(retrieved_docs)} documents for usefulness to query: {query}")
    result = list()
    
    
    message = [{"role":"system", "content":eval_query},
               {"role":"user", "content":(f"query: {query}\n "
                         f"document: {retrieved_docs}\n")}
                ]
    response = client.chat.completions.create(
                model='gpt-4o-mini',
                messages=message,
                temperature=0.2)

    result.append(response.choices[0].message.content)

    return result
     

#### Game Web Search Tool

In [379]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 
class GameWebSearchResults(BaseModel):
    """Representation of results from a web search"""
    title: str
    url: str
    content: str
    score: float

@tool
def game_web_search_tool(query: str) -> List[dict]:
    """
    Based on the user's query about the game industry perform a web search. 
   args: 
   - query (str): game industry query from user
   The result includes:
       List[GameWebSearchResults]: list object containing a dictionary of results from a web search, including:
           title: The title of the search result.
           url: The URL of the search result.
           content: A snippet of the most query-related content extracted from the scraped URL.
           score: A relevance score for the result."""

    print('Using Tool: **game_web_search_tool**')
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    search_depth = "advanced"
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    output= {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results",[]),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    out: List[Dict] = []
    for o in output.get("results",[]):
        out.append(json.dumps({"title":o.get("title",""),
                    "url": o.get("url",""),
                    "content": o.get("content"),
                    "score": o.get("score")}))
    return {'answer':out}    

### Final Answer Tool

In [380]:
class final_answer(BaseModel):
    """Representation of final answer and where the answer came from"""
    answer: str
    citations: dict

@tool
def final_answer_tool(final_response: str, citations:dict) -> dict:
    """Returns final response and the citations to the query.
    Args: 
        final_response: final human readable answer.
        citations: dict of sources as either file title or the website it was retrieved from. Format should either be 'internal database object on %name of game%', or '%site name%: http'
    Returns:
        {'answer': {"final_response": final_response, "citations":citations}}
    """
    return {'answer': {"final_response": final_response, "citations":citations}}
    

### Agent

In [407]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
from lib.tooling import Tool, ToolCall, Union
from typing import TypedDict, Optional
from lib.state_machine import StateMachine, Run, EntryPoint, Step, Termination
import json
from lib.memory import ShortTermMemory
tool_list = [game_web_search_tool,evaluate_retrieval_tool, retrieve_game_tool, final_answer_tool]

class AgentState(TypedDict):
    user_query: str  # The current user query being processed
    instructions: str  # System instructions for the agent
    messages: List[dict]  # List of conversation messages
    current_tool_calls: Optional[List[ToolCall]]  # Current pending tool calls
    total_tokens: int

class Agent:
    def __init__(self, 
                 model_name: str,
                 instructions: str, 
                 tools: List[Tool] = None,
                 temperature: float = 0.7):
        """
        Initialize an Agent instance
        
        Args:
            model_name: Name/identifier of the LLM model to use
            instructions: System instructions for the agent
            tools: Optional licontentst of tools available to the agent
            temperature: Temperature parameter for LLM (default: 0.7)
        """
        self.instructions = instructions
        self.tools = tools if tools else []
        self.model_name = model_name
        self.temperature = temperature
        self.memory = ShortTermMemory()
                
        # Initialize state machine
        self.workflow = self._create_state_machine()

    def _prepare_messages_step(self, state: AgentState) -> AgentState:
        """Step logic: Prepare messages for LLM consumption"""
        messages = state.get("messages", [])
            
        # If no messages exist, start with system message
        if not messages:
            messages = [SystemMessage(content=state["instructions"])]
            
        # Add the new user message
        messages.append(UserMessage(content=state["user_query"]))
        
        return {
            "messages": messages,
            "session_id": state["session_id"]
        }

    def _llm_step(self, state: AgentState) -> AgentState:
        """Step logic: Process the current state through the LLM"""
        openai_api_key = os.getenv("OPENAI_API_KEY")
        # Initialize LLM
        llm = LLM(
            model=self.model_name,
            temperature=self.temperature,
            tools=self.tools,
            api_key=openai_api_key,
            base_url="https://openai.vocareum.com/v1"
        )

        response = llm.invoke(state["messages"])
        tool_calls = response.tool_calls if response.tool_calls else None
        
        current_total = state.get("total_tokens", 0)
        if response.token_usage:
            current_total += response.token_usage.total_tokens
            
        # Create AI message with content and tool calls
        ai_message = AIMessage(content=response.content, tool_calls=tool_calls)
        
        
        return {
            "messages": state["messages"] + [ai_message],
            "current_tool_calls": tool_calls,
            "session_id": state["session_id"],
            "total_tokens": current_total,
        }

    def _tool_step(self, state: AgentState) -> AgentState:
        """Step logic: Execute any pending tool calls"""
        tool_calls = state["current_tool_calls"] or []
        tool_messages = []
        
        for call in tool_calls:
            # Access tool call data correctly
            function_name = call.function.name
            function_args = json.loads(call.function.arguments)
            tool_call_id = call.id
            # Find the matching tool
            tool = next((t for t in self.tools if t.name == function_name), None)
            if tool:
                result = tool(**function_args)
                tool_messages.append(
                    ToolMessage(
                        content=json.dumps(result), 
                        tool_call_id=tool_call_id, 
                        name=function_name, 
                    )
                )
        
        # Clear tool calls and add results to messages
        return {
            "messages": state["messages"] + tool_messages,
            "current_tool_calls": None,
            "session_id": state["session_id"]
        }

    def _create_state_machine(self) -> StateMachine[AgentState]:
        """Create the internal state machine for the agent"""
        machine = StateMachine[AgentState](AgentState)
        
        # Create steps
        entry = EntryPoint[AgentState]()
        message_prep = Step[AgentState]("message_prep", self._prepare_messages_step)
        llm_processor = Step[AgentState]("llm_processor", self._llm_step)
        tool_executor = Step[AgentState]("tool_executor", self._tool_step)
        termination = Termination[AgentState]()
        
        machine.add_steps([entry, message_prep, llm_processor, tool_executor, termination])
        
        # Add transitions
        machine.connect(entry, message_prep)
        machine.connect(message_prep, llm_processor)
        
        # Transition based on whether there are tool calls
        def check_tool_calls(state: AgentState) -> Union[Step[AgentState], str]:
            """Transition logic: Check if there are tool calls"""
            if state.get("current_tool_calls"):
                return tool_executor
            return termination
        
        machine.connect(llm_processor, [tool_executor, termination], check_tool_calls)
        machine.connect(tool_executor, llm_processor)  # Go back to llm after tool execution
        
        return machine

    def invoke(self, query: str,  session_id: Optional[str] = None) -> Run:
        """
        Run the agent on a query
        
        Args:
            query: The user's query to process
            session_id: Optional session identifier (uses "default" if None)
            
        Returns:
            The final run object after processing
        """
        session_id = session_id or "default"

        # Create session if it doesn't exist
        self.memory.create_session(session_id)

        # Get previous messages from last run if available
        previous_messages = []
        last_run: Run = self.memory.get_last_object(session_id)
        if last_run:
            last_state = last_run.get_final_state()
            if last_state:
                previous_messages = last_state["messages"]
        
        initial_state: AgentState = {
            "user_query": query,
            "instructions": self.instructions,
            "messages": previous_messages,
            "current_tool_calls":None,
            "session_id":session_id
        }

        run_object = self.workflow.run(initial_state)

        self.memory.add(run_object, session_id)

        return run_object



game_agent = Agent(
    model_name="gpt-4o-mini",
    tools=tool_list,
    instructions=(
        "You're an AI Agent very good at ideo game industry research "
        "You can answer multistep questions by sequentially calling functions. "
        "You follow a pattern of of Thought and Action. "
        "Last tool used is always  **final_answer_tool** when usable document or information from web search is recieved from tool call.\n" 
        "Create a plan of execution: "
        "- Use Thought to describe your thoughts about the question you have been asked. "
        "- Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly."
        "When you think it's over, return the answer. Never try to respond directly if the question needs a tool. "
        "But if you don't have a tool available, you can respond directly. "
        "Tools Included: \n"
        " - **game_web_search_tool**: Searches the web for supplimentary information about a video game.\n"
        " - **evaluate_retrieval_tool**: Evaluates relevance of the retrieved documents from the retrieve_game_tool.\n"
        " - **retrieve_game_tool**: Searches an internal collection for relevant documents about a video game.\n"
        " - **final_answer_tool**: Provides the final answer to the user's query and ends the agent process.\n"
        "\n"
        " Tool use instructions:\n"
        "    1. Use **retrieve_game_tool** to search for information to answer a specific query\n"
        "    2. Use **evaluate_retrieval_tool** to evaluate how well the retrieved documents in step 1 address the user query. DO NOT USE FOR **game_web_search_tool** RESULTS!\n"
        "    3. If Step 2 provides a useful document to anwer the question, then call **final_answer_tool**, otherwise, continue.\n"
        "    3. Use **game_web_search_tool** to search the internet if the documents retrieved were not deemed useful in the evaluation provided by evaluate_retrieval_tool.\n"
        "    4. If web search did not return relevant results, try **game_web_search_tool** again with a slightly modified version of the user question.\n"
        "    5. If web search provided relevant results call **final_answer_tool**\n" 
        "\n"    
    )
)



In [382]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
result_1 = game_agent.invoke(query="When Pokémon Gold and Silver was released?")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Using Tool: **retrieve_game_tool**
Looking for documents to address query: Pokémon Gold and Silver release date
3 documents found.
______________ Documents Related to: Pokémon Gold and Silver release date _______________
['Platform: Game Boy Color\n, Name: Pokémon Gold and Silver\n, YearOfRelease: 1999\n, Description: Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.\n, SimilarityScore: 0.87\n', 'Platform: Game Boy Advance\n, Name: Pokémon Ruby and Sapphire\n, YearOfRelease: 2002\n, Description: Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles.\n, SimilarityScore: 0.84\n', 'Platform: Nintendo Switch\n, Name: Mario Kart 8 Deluxe\n, YearOfRelease: 2017\n, Description: An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.\n, Simila

In [383]:
print(result_1.get_final_state()["messages"])

[SystemMessage(role='system', content="You're an AI Agent very good at ideo game industry research You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Last tool used is always  **final_answer_tool** when usable document or information from web search is recieved from tool call.\nCreate a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly.When you think it's over, return the answer. Never try to respond directly if the question needs a tool. But if you don't have a tool available, you can respond directly. Tools Included: \n - **game_web_search_tool**: Searches the web for supplimentary information about a video game.\n - **evaluate_retrieval_tool**: Evaluates relevance of the retrieved documents from the retrieve_game_tool.\n - **retrieve_game_tool**: Searc

In [400]:
result_1.get_final_state()["messages"][-2].content

'{"answer": {"final_response": "Pok\\u00e9mon Gold and Silver were released in 1999.", "citations": "internal database object on Pok\\u00e9mon Gold and Silver"}}'

In [385]:
# - Which one was the first 3D platformer Mario game?
result_2 = game_agent.invoke(query="Which one was the first 3D platformer Mario game?")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Using Tool: **retrieve_game_tool**
Looking for documents to address query: first 3D platformer Mario game
3 documents found.
______________ Documents Related to: first 3D platformer Mario game _______________
["Platform: Nintendo 64\n, Name: Super Mario 64\n, YearOfRelease: 1996\n, Description: A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.\n, SimilarityScore: 0.9\n", 'Platform: Super Nintendo Entertainment System (SNES)\n, Name: Super Mario World\n, YearOfRelease: 1990\n, Description: A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.\n, SimilarityScore: 0.87\n', 'Platform: Nintendo Switch\n, Name: Mario Kart 8 Deluxe\n, YearOfRelease: 2017\n, Description: An enhanced version of Mario Kart 8, featuring new characters, tracks, and impr

In [386]:
print(result_2.get_final_state()["messages"])

[SystemMessage(role='system', content="You're an AI Agent very good at ideo game industry research You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Last tool used is always  **final_answer_tool** when usable document or information from web search is recieved from tool call.\nCreate a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly.When you think it's over, return the answer. Never try to respond directly if the question needs a tool. But if you don't have a tool available, you can respond directly. Tools Included: \n - **game_web_search_tool**: Searches the web for supplimentary information about a video game.\n - **evaluate_retrieval_tool**: Evaluates relevance of the retrieved documents from the retrieve_game_tool.\n - **retrieve_game_tool**: Searc

In [401]:
result_2.get_final_state()['messages'][-2].content

'{"answer": {"final_response": "The first 3D platformer Mario game is \'Super Mario 64\', released in 1996. It was a groundbreaking title that set new standards for the genre.", "citations": "internal database object on Super Mario 64"}}'

In [388]:
# - Was Mortal Kombat X realeased for Playstation 5?
result_3 = game_agent.invoke(query="Was Mortal Kombat X realeased for Playstation 5?")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Using Tool: **retrieve_game_tool**
Looking for documents to address query: Mortal Kombat X release for Playstation 5
3 documents found.
______________ Documents Related to: Mortal Kombat X release for Playstation 5 _______________
["Platform: PlayStation 5\n, Name: Marvel's Spider-Man 2\n, YearOfRelease: 2023\n, Description: The sequel to the acclaimed Spider-Man game, featuring both Peter Parker and Miles Morales as playable characters.\n, SimilarityScore: 0.82\n", "Platform: Xbox Series X|S\n, Name: Halo Infinite\n, YearOfRelease: 2021\n, Description: The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting.\n, SimilarityScore: 0.8\n", "Platform: PlayStation 4\n, Name: Marvel's Spider-Man\n, YearOfRelease: 2018\n, Description: An open-world superhero game that lets players swing through New York City as Spider-Man

In [389]:
print(result_3.get_final_state()["messages"])

[SystemMessage(role='system', content="You're an AI Agent very good at ideo game industry research You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Last tool used is always  **final_answer_tool** when usable document or information from web search is recieved from tool call.\nCreate a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly.When you think it's over, return the answer. Never try to respond directly if the question needs a tool. But if you don't have a tool available, you can respond directly. Tools Included: \n - **game_web_search_tool**: Searches the web for supplimentary information about a video game.\n - **evaluate_retrieval_tool**: Evaluates relevance of the retrieved documents from the retrieve_game_tool.\n - **retrieve_game_tool**: Searc

In [402]:
result_3.get_final_state()['messages'][-2].content

'{"answer": {"final_response": "Mortal Kombat X was not released for PlayStation 5. It was originally released for PlayStation 4 and Xbox One on April 14, 2015. While there are other Mortal Kombat titles and collections being released for PS5, Mortal Kombat X itself is not among them.", "citations": "Wikipedia: https://en.wikipedia.org/wiki/Mortal_Kombat_X"}}'

In [391]:
# - who made halo infinite?
result_4 = game_agent.invoke(query="who made halo infinite?")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Using Tool: **retrieve_game_tool**
Looking for documents to address query: who made Halo Infinite
3 documents found.
______________ Documents Related to: who made Halo Infinite _______________
["Platform: Xbox Series X|S\n, Name: Halo Infinite\n, YearOfRelease: 2021\n, Description: The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting.\n, SimilarityScore: 0.82\n", 'Platform: Xbox One\n, Name: Minecraft\n, YearOfRelease: 2014\n, Description: A sandbox game that allows players to build and explore infinite worlds, fostering creativity and adventure.\n, SimilarityScore: 0.72\n', "Platform: PlayStation 5\n, Name: Marvel's Spider-Man 2\n, YearOfRelease: 2023\n, Description: The sequel to the acclaimed Spider-Man game, featuring both Peter Parker and Miles Morales as playable characters.\n, SimilarityScore: 0.71\n"]
[S

In [392]:
print(result_4.get_final_state()["messages"])

[SystemMessage(role='system', content="You're an AI Agent very good at ideo game industry research You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Last tool used is always  **final_answer_tool** when usable document or information from web search is recieved from tool call.\nCreate a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly.When you think it's over, return the answer. Never try to respond directly if the question needs a tool. But if you don't have a tool available, you can respond directly. Tools Included: \n - **game_web_search_tool**: Searches the web for supplimentary information about a video game.\n - **evaluate_retrieval_tool**: Evaluates relevance of the retrieved documents from the retrieve_game_tool.\n - **retrieve_game_tool**: Searc

In [403]:
result_4.get_final_state()['messages'][-2].content

'{"answer": {"final_response": "Halo Infinite was developed by 343 Industries, with assistance from several other studios including SkyBox Labs, Sperasoft, The Coalition, Certain Affinity, and Atomhawk.", "citations": "Wikipedia: https://en.wikipedia.org/wiki/Halo_Infinite"}}'

### Statefull quetion Responses

In [408]:
result_5 = game_agent.invoke(query="What was the biggest game release of all time?", session_id='big_game')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Using Tool: **game_web_search_tool**
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
Using Tool **evaluate_retrieval_tool**
Evaluating 5 documents for usefulness to query: biggest game release of all time
Using Tool **evaluate_retrieval_tool**
Evaluating 2 documents for usefulness to query: biggest game release of all time
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [409]:
result_5.get_final_state()['messages'][-2].content

'{"answer": {"final_response": "The biggest game release of all time, based on sales figures, is **Minecraft**, which has sold over 350 million copies as of October 2023. In terms of gross revenue, **Grand Theft Auto V** is the highest-grossing video game, having generated over $6 billion in revenue since its release.", "citations": {"1": "Wikipedia: https://en.wikipedia.org/wiki/List_of_best-selling_video_games", "2": "IGN: https://www.ign.com/articles/best-selling-video-games-of-all-time-grand-theft-auto-minecraft"}}}'

In [410]:
result_6 = game_agent.invoke(query="What was the second?", session_id='big_game')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Using Tool: **game_web_search_tool**
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
Using Tool **evaluate_retrieval_tool**
Evaluating 3 documents for usefulness to query: second biggest game release of all time
Using Tool **evaluate_retrieval_tool**
Evaluating 2 documents for usefulness to query: second biggest game release of all time
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [411]:
result_6.get_final_state()['messages']

[SystemMessage(role='system', content="You're an AI Agent very good at ideo game industry research You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Last tool used is always  **final_answer_tool** when usable document or information from web search is recieved from tool call.\nCreate a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly.When you think it's over, return the answer. Never try to respond directly if the question needs a tool. But if you don't have a tool available, you can respond directly. Tools Included: \n - **game_web_search_tool**: Searches the web for supplimentary information about a video game.\n - **evaluate_retrieval_tool**: Evaluates relevance of the retrieved documents from the retrieve_game_tool.\n - **retrieve_game_tool**: Searc

In [412]:
result_6.get_final_state()['messages'][-2].content

'{"answer": {"final_response": "The second biggest game release of all time is **Grand Theft Auto V**, with over 215 million copies sold as of October 2023. It is also recognized as the highest-grossing video game in terms of revenue, exceeding $6 billion.", "citations": {"1": "Wikipedia: https://en.wikipedia.org/wiki/List_of_best-selling_video_games", "2": "GameSpot: https://www.gamespot.com/gallery/top-10-best-selling-video-games-of-all-time/2900-4814/"}}}'

### (Optional) Advanced

In [398]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes